In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score,recall_score


In [ ]:
npzfile=np.load('/content/drive/MyDrive/Cigerate/cigerate_triplet .npz')
image_pair=npzfile['arr_0']
image_label=npzfile['arr_1']

In [ ]:
len(image_pair)

2744

In [ ]:
def main(thresh):
  
  t0 = time.time()


  def CalcFeatures(img, th):
    sift = cv2.xfeatures2d.SIFT_create(th)
    kp, des = sift.detectAndCompute(img, None)
    return des
  
  '''
  All the files appended to the image_path list are passed through the
  CalcFeatures functions which returns the descriptors which are 
  appended to the features list and then stacked vertically in the form
  of a numpy array.
  '''

  features = []
  for i in range(len(image_pair)):
    
    img_des = CalcFeatures(cv2.normalize(image_pair[i][1], None, 0, 255, cv2.NORM_MINMAX).astype('uint8'),thresh )
    if img_des is not None:
        features.append(img_des)
        
  features = np.vstack(features)

  '''
  K-Means clustering is then performed on the feature array obtained 
  from the previous step. The centres obtained after clustering are 
  further used for bagging of features.
  '''

  k = len(set(image_label.squeeze()))*20
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.1) #It is the iteration termination criteria. 
  flags = cv2.KMEANS_RANDOM_CENTERS
  compactness, labels, centres = cv2.kmeans(features, k, None, criteria, 10, flags)

  '''
  The bag_of_features function assigns the features which are similar
  to a specific cluster centre thus forming a Bag of Words approach.  
  '''

  def bag_of_features(features, centres, k = 500):
      vec = np.zeros((1, k))
      for i in range(features.shape[0]):
          feat = features[i]
          diff = np.tile(feat, (k, 1)) - centres
          dist = pow(((pow(diff, 2)).sum(axis = 1)), 0.5)
          idx_dist = dist.argsort()
          idx = idx_dist[0]
          vec[0][idx] += 1
      return vec

  vec = []
  labels=[]
  for i in range(len(image_pair)):
    
    img_des = CalcFeatures(cv2.normalize(image_pair[i][1], None, 0, 255, cv2.NORM_MINMAX).astype('uint8'),thresh)
    if img_des is not None:
        img_vec = bag_of_features(img_des, centres, k)
        vec.append(img_vec)
        labels.append(image_label[i])
  vec = np.vstack(vec)
  
  '''
  Splitting the data formed into test and split data and training the 
  SVM Classifier.
  '''

  X_train, X_test, y_train, y_test = train_test_split(vec, np.array(labels).ravel(), test_size=0.33, stratify=np.array(labels).ravel())
  clf = SVC()
  clf.fit(X_train, y_train)
  preds = clf.predict(X_test)
  acc = accuracy_score(y_test, preds)
  conf_mat = confusion_matrix(y_test, preds)
  preci=precision_score(y_test, preds,average='macro')
  recall=recall_score(y_test, preds,average='macro')


  t1 = time.time()
  
  return acc*100, conf_mat, preci*100,recall*100,(t1-t0)


accuracy = []
timer = []
recall=[]
precision=[]
for i in range(20,80,5):
  print('\nCalculating for a threshold of {}'.format(i))
  data = main(i)
  accuracy.append(data[0])
  precision.append(data[2])
  recall.append(data[3])
  conf_mat = data[1]
  timer.append(data[4])
  print('\nAccuracy = {}\nTime taken = {} sec\nConfusion matrix :\n{} \nprecision :\n{} \nrecall :\n{}'.format(data[0],data[4],data[1],data[2],data[3]))


Calculating for a threshold of 20

Accuracy = 85.98233995584988
Time taken = 71.15980100631714 sec
Confusion matrix :
[[ 89  10   0   0   0   0   1   0   0   0]
 [  2 318   0   6   1   0   2   0   0   0]
 [  0   6  13   1   1   0   1   0   0   0]
 [  0  17   0 115   0   0   4   0   0   0]
 [  0  16   0   2  16   4   0   0   0   0]
 [  1  13   0   0   0  44   4   1   0   0]
 [  0   9   1   2   0   1  90   0   0   0]
 [  0   5   0   0   1   3   0  55   0   0]
 [  0   2   0   0   1   2   0   0  21   0]
 [  0   5   0   0   0   1   1   0   0  18]] 
precision :
90.57607866886616 
recall :
76.73381721198226

Calculating for a threshold of 25

Accuracy = 87.85871964679912
Time taken = 81.74209570884705 sec
Confusion matrix :
[[ 90   9   0   0   0   1   0   0   0   0]
 [  2 323   0   0   0   2   2   0   0   0]
 [  0   8  10   2   0   0   2   0   0   0]
 [  1  17   0 117   0   0   1   0   0   0]
 [  0  19   0   1  15   3   0   0   0   0]
 [  1  10   0   0   0  51   1   0   0   0]
 [  0  12   0 